In [1]:
#Make imports
# !pip install fuzzywuzzy
# !pip install python-Levenshtein
# !pip install sklearn
# !pip install pandas
# !pip install numpy

import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import string
import sklearn
import numpy as np
import time
from enum import Enum

!pip install multiprocess
import multiprocess
import multiprocessing as mp
from multiprocessing import Pool, Process


import nltk
from nltk import pos_tag, pos_tag_sents
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
stop_words = set(stopwords.words('english')) - set(['at', 'do', 'your', 'from', 'to', 'out', 'no', 'the'])

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/priyanshkedia/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/priyanshkedia/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
start = time.time()
path = '/Users/priyanshkedia/Documents/Machine Learning/Twitter-Analysis/Latest/output_data/Tue Aug 18/combined_Tue Aug 18.csv'
# path = '/home/manikya_varshney/Documents/Python/Yale/Latest/output_data/Tue Aug 18/combined_Tue Aug 18.csv'
data = pd.read_csv(path, index_col=None, header=0, engine='python' )
end = time.time()
print("Read csv with pandas: ",(end-start),"sec")

Read csv with pandas:  4.438261032104492 sec


In [3]:
keywords_Imp = ['stay at home' , 'do your part', 'Responsible', 
            'home', 'house', 'cancel', 'shutdown', 'postpone',
            'school closure', 'Closure', 'business closure',
            'suspension', 'quarantine', 'lockdown', 'social distance', 
            'social distancing', 'self quarantine', 'isolat', '6-feet',
            'distance', '#clubquarantine', '#quarantinelife', '#quarantineacitivites']

keywords_Ada = ['school from home' , 'learn', 'remote', 'school food service', 
            'online shopping', 'online purchase', 'online church', 'delivery',
            'drive thru', 'to go', 'take out', 'Tiktok', 'Netflix', 'telework', 
            'zoom', 'telehealth', 'telemedicine', 'work from home', 'wfh',
            'working at home', 'working remotely', 'online meeting']

keywords_Ne = ['bored' , 'lonely', 'stress', 
            'anxiety', 'scared', 'worry', 'end', 'cabin fever',
            '#sideeffectsofquarantinelife', 'tissue paper', 'toilet paper']

keywords_Sd = ['social functions' , 'gathering', 'empty streets', 
            'interaction', 'large', 'no cars', 'non-essential',
            'travel', 'unnecessary', 'crowd']

keywords_Purp = ['Flatten the curve' , 'Slow the spread', 'slow transmission', 
            'protect', 'save', '#stayhomesavelives']

keywords_Pe = ['silver lining' , 'optimistic', 'hope', 
            'bright side', 'Safe', '#togetherapart']

In [4]:
# Constants
num_cores = mp.cpu_count()
FINAL_COL_NAME = "FINAL_TEXT"

In [5]:
def keywords_cleaning(keywords_list):
    
    #Convert to lower
    for i in range(len(keywords_list)): 
        keywords_list[i] = keywords_list[i].lower()
    
    #Remove punctuations
    for i in range(len(keywords_list)):
        keywords_list[i] = keywords_list[i].translate(str.maketrans('','',string.punctuation))
    
    #More cleaning
    for i in range(len(keywords_list)):
        keywords_list[i] = keywords_list[i].replace('/[^a-zA-Z0-9 ]/g', '').replace('\n',' ').strip('“').strip('“').strip('’').lstrip(' ').rstrip(' ')

    #Remove stop words
    def remove_stopwords(data):
        output_array=[]
        for sentence in data:
            temp_list=[]
            for word in sentence.split():
                if word not in stop_words:
                    temp_list.append(word)
            output_array.append(' '.join(temp_list))
        return output_array

    keywords_list=remove_stopwords(keywords_list)

    #Stemming
    ps = PorterStemmer()
    keywords_list_stem = [[ps.stem(word) for word in sentence.split(" ")] for sentence in keywords_list]
    keywords_list_final = [" ".join(sentence) for sentence in keywords_list_stem]

    return keywords_list_final


In [6]:
# Enum for facets
class Facets(Enum):
    IMPLEMENTATION = "Imp"
    ADAPTATION = "Ada"
    NEGATIVE_EMOTIONS = "Ne"
    SOCIAL_DISRUPTION = "Sd"
    PURPOSE = "Purp"
    POSITIVE_EMOTION = "Pe"

In [32]:
def fuzzy_logic(row, FINAL_COL_NAME, keywords, facet):
    keyword_match, score = process.extractOne(row[FINAL_COL_NAME], keywords, scorer = fuzz.partial_ratio)
    row['final_score'] = score
    row['final_keyword_match'] = keyword_match
    return row

In [42]:
def keep_only_highest(data, high_value, facet):
    data['final_score'] = data['final_score'].astype(int)    
    data = data[data['final_score'] == 100].reset_index(drop=True)  
    data['FACET'] = facet.value 
    return data

In [43]:
def proportion(interim_data_final, interim_data):
    numerator = interim_data_final.shape[0]
    denominator = interim_data.shape[0]
    prop_val = (numerator/denominator)
    return prop_val

In [44]:
def split_dataframe(df, nums = 4): 
    chunks = list()
    num_chunks = nums
    chunk_size = len(df) // nums
    for i in range(num_chunks):
        chunks.append(df[i*chunk_size:(i+1)*chunk_size])
    return chunks

In [45]:
def multiprocessing_(data_, keywords_, facet_):
    print(data_.shape)
    start = time.time()
    pool = Pool(num_cores)
    df = split_dataframe(data_, num_cores)
    data1 = pool.starmap(fuzzy_logic, [(x, FINAL_COL_NAME, keywords_, facet_) for x in df])
    pool.close()
    pool.join()
    end = time.time()
    print("time = ", end - start)
    return data1

### Keywords Cleaning

In [46]:
keywords_Imp = keywords_cleaning(keywords_Imp)
keywords_Ada = keywords_cleaning(keywords_Ada)
keywords_Ne = keywords_cleaning(keywords_Ne)
keywords_Sd = keywords_cleaning(keywords_Sd)
keywords_Purp = keywords_cleaning(keywords_Purp)
keywords_Pe = keywords_cleaning(keywords_Pe)

### Drop Rows with Empty Tweets

In [47]:
nan_value = float("NaN")
data[FINAL_COL_NAME].replace("", nan_value, inplace=True)
data.dropna(subset = [FINAL_COL_NAME], inplace=True)

data = data.head(100)

# 1_Implementation

In [48]:
#interim_Imp = multiprocessing_(data, keywords_Imp, Facets.IMPLEMENTATION)
interim_Imp = data.apply(fuzzy_logic, axis = 1, args = (FINAL_COL_NAME, keywords_Imp, Facets.IMPLEMENTATION))
interim_Imp_final = keep_only_highest(interim_Imp, 100, Facets.IMPLEMENTATION)

In [49]:
interim_Imp_final

,created_at,id,source,is_quote_tweet,quoted_tweet_id,in_reply_to_status_id_str,in_reply_to_user_id_str,in_reply_to_screen_name,lang,quote_count,...,QT_place_country_code,QT_coordinates,QT_text,QT_full_text,TEST_FLAG,RT,FINAL_TEXT,final_score,final_keyword_match,FACET
0,Tue Aug 18 20:16:16 +0000 2020,1.295816806842749e+18,"<a href=""http://twitter.com/download/iphone"" r...",False,NaN,1.295808857885868e+18,204458518.0,briduimhaoluala,en,0.0,...,NaN,NaN,NaN,NaN,Y,False,briduimhaoluala focu highrisk member mere unio...,100,social distanc,Imp
1,Tue Aug 18 20:16:16 +0000 2020,1.2958168079290737e+18,"<a href=""http://instagram.com"" rel=""nofollow"">...",False,NaN,NaN,NaN,NaN,en,0.0,...,NaN,NaN,NaN,NaN,Y,False,today mission tough quarantinelif keep kill wi...,100,quarantin,Imp
2,Tue Aug 18 20:16:16 +0000 2020,1.2958168082771722e+18,"<a href=""http://twitter.com/download/iphone"" r...",False,NaN,NaN,NaN,NaN,en,0.0,...,NaN,NaN,NaN,NaN,Y,False,sampp 500 close at record high eras the last s...,100,shutdown,Imp
3,Tue Aug 18 20:16:17 +0000 2020,1.2958168101813327e+18,"<a href=""http://twitter.com/download/iphone"" r...",False,NaN,1.2958154284308685e+18,3162911248.0,mcm_ct,en,0.0,...,NaN,NaN,NaN,NaN,Y,False,mcmct anyon spend 2000 ppe shouldnt teach anyon,100,hou,Imp
4,Tue Aug 18 20:16:17 +0000 2020,1.2958168118465413e+18,"<a href=""http://twitter.com/download/iphone"" r...",False,NaN,NaN,NaN,NaN,en,0.0,...,NaN,NaN,NaN,NaN,Y,False,caught glimps someon wear dangl ear mask thoug...,100,hou,Imp
5,Tue Aug 18 20:16:18 +0000 2020,1.2958168135324672e+18,"<a href=""http://twitter.com/download/android"" ...",False,NaN,1.295784486949462e+18,7.810050522070385e+17,MyJoze,en,0.0,...,NaN,NaN,NaN,NaN,Y,False,myjoz trump elimin the entir pandem respons team,100,respon,Imp
6,Tue Aug 18 20:16:18 +0000 2020,1.29581681317607e+18,"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,NaN,en,0.0,...,NaN,NaN,NaN,NaN,Y,False,at home took time out to go to art school part...,100,home,Imp
7,Tue Aug 18 20:16:18 +0000 2020,1.2958168153864028e+18,"<a href=""https://mobile.twitter.com"" rel=""nofo...",False,NaN,1.2958148671448023e+18,1.098415694021124e+18,collegemasc,en,0.0,...,NaN,NaN,NaN,NaN,Y,False,collegemasc 1 start septemb would spread globa...,100,quarantin,Imp
8,Tue Aug 18 20:16:18 +0000 2020,1.295816816883835e+18,"<a href=""http://twitter.com/download/iphone"" r...",False,NaN,1.2957868603300618e+18,7.986988832494756e+17,rickgtz22,en,0.0,...,NaN,NaN,NaN,NaN,Y,False,rickgtz22 richard do prevent death do lock for...,100,respon,Imp
9,Tue Aug 18 20:16:19 +0000 2020,1.295816819010306e+18,"<a href=""http://twitter.com/download/iphone"" r...",False,NaN,1.2958046937430467e+18,1673526829.0,md_mszubairi,en,0.0,...,NaN,NaN,NaN,NaN,Y,False,mdmszubairi share concern run real risk creat ...,100,respon,Imp


# 2_Adaptation

In [50]:
interim_Ada = data.apply(fuzzy_logic, axis = 1, args = (FINAL_COL_NAME, keywords_Ada, Facets.ADAPTATION))
interim_Ada_final = keep_only_highest(interim_Ada, 100, Facets.ADAPTATION)

In [51]:
interim_Ada_final

,created_at,id,source,is_quote_tweet,quoted_tweet_id,in_reply_to_status_id_str,in_reply_to_user_id_str,in_reply_to_screen_name,lang,quote_count,...,QT_place_country_code,QT_coordinates,QT_text,QT_full_text,TEST_FLAG,RT,FINAL_TEXT,final_score,final_keyword_match,FACET
0,Tue Aug 18 20:16:16 +0000 2020,1.295816806263816e+18,"<a href=""http://twitter.com/download/iphone"" r...",False,NaN,1.2957952123551662e+18,288220068.0,CarolineHutt,en,0.0,...,NaN,NaN,NaN,NaN,Y,False,carolinehutt massiv leg let see learn multipl ...,100,learn,Ada
1,Tue Aug 18 20:16:18 +0000 2020,1.29581681317607e+18,"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,NaN,en,0.0,...,NaN,NaN,NaN,NaN,Y,False,at home took time out to go to art school part...,100,to go,Ada
2,Tue Aug 18 20:16:22 +0000 2020,1.2958168298273137e+18,"<a href=""https://mobile.twitter.com"" rel=""nofo...",False,NaN,NaN,NaN,NaN,en,0.0,...,NaN,NaN,NaN,NaN,Y,False,less 10 the agenc particip the 2020 studi said...,100,remot,Ada
3,Tue Aug 18 20:16:24 +0000 2020,1.2958168417392845e+18,"<a href=""http://twitter.com/download/android"" ...",False,NaN,1.2954002444658524e+18,605727932.0,pamela12632553,en,0.0,...,NaN,NaN,NaN,NaN,Y,False,pamela12632553 theyr to cover ass need to go t...,100,to go,Ada
4,Tue Aug 18 20:16:25 +0000 2020,1.2958168441510666e+18,"<a href=""https://mobile.twitter.com"" rel=""nofo...",False,NaN,NaN,NaN,NaN,en,0.0,...,NaN,NaN,NaN,NaN,Y,False,covid19 morph excus overreach at mani level ch...,100,zoom,Ada


# 3_Negative Emotion

In [52]:
interim_Ne = data.apply(fuzzy_logic, axis = 1, args = (FINAL_COL_NAME, keywords_Ne, Facets.NEGATIVE_EMOTIONS))
interim_Ne_final = keep_only_highest(interim_Ne, 100, Facets.NEGATIVE_EMOTIONS)

In [53]:
interim_Ne_final

,created_at,id,source,is_quote_tweet,quoted_tweet_id,in_reply_to_status_id_str,in_reply_to_user_id_str,in_reply_to_screen_name,lang,quote_count,...,QT_place_country_code,QT_coordinates,QT_text,QT_full_text,TEST_FLAG,RT,FINAL_TEXT,final_score,final_keyword_match,FACET
0,Tue Aug 18 20:16:17 +0000 2020,1.2958168101813327e+18,"<a href=""http://twitter.com/download/iphone"" r...",False,NaN,1.2958154284308685e+18,3162911248.0,mcm_ct,en,0.0,...,NaN,NaN,NaN,NaN,Y,False,mcmct anyon spend 2000 ppe shouldnt teach anyon,100,end,Ne
1,Tue Aug 18 20:16:22 +0000 2020,1.2958168329983508e+18,"<a href=""http://twitter.com/download/iphone"" r...",False,NaN,1.295687488279851e+18,466420346.0,Eskom_SA,en,0.0,...,NaN,NaN,NaN,NaN,Y,False,eskomsa 6 month corona stress south african ca...,100,stress,Ne
2,Tue Aug 18 20:16:26 +0000 2020,1.295816848815149e+18,"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,NaN,en,0.0,...,NaN,NaN,NaN,NaN,Y,False,thousand attend pool parti wuhan china the cit...,100,end,Ne
3,Tue Aug 18 20:16:26 +0000 2020,1.2958168503122903e+18,"<a href=""https://mobile.twitter.com"" rel=""nofo...",False,NaN,NaN,NaN,NaN,en,0.0,...,NaN,NaN,NaN,NaN,Y,False,everyon quarantin bore armi yike suck to guess,100,bore,Ne
4,Tue Aug 18 20:16:27 +0000 2020,1.295816851998544e+18,"<a href=""http://twitter.com/download/iphone"" r...",False,NaN,NaN,NaN,NaN,en,0.0,...,NaN,NaN,NaN,NaN,Y,False,feel like friend quarantin get test liter scar...,100,scare,Ne
5,Tue Aug 18 20:16:29 +0000 2020,1.2958168617922478e+18,"<a href=""https://mobile.twitter.com"" rel=""nofo...",False,NaN,NaN,NaN,NaN,en,0.0,...,NaN,NaN,NaN,NaN,Y,False,longsuff sar patient offer clue the worrisom f...,100,worri,Ne
6,Tue Aug 18 20:16:31 +0000 2020,1.2958168685491118e+18,"<a href=""https://mobile.twitter.com"" rel=""nofo...",False,NaN,1.2950070528305766e+18,1.2358497971914015e+18,ProfKarolSikora,en,0.0,...,NaN,NaN,NaN,NaN,Y,False,profkarolsikora un agenda21 depopul no covid,100,end,Ne


# 4_Social Disruption

In [54]:
interim_Sd = data.apply(fuzzy_logic, axis = 1, args = (FINAL_COL_NAME, keywords_Sd, Facets.SOCIAL_DISRUPTION))
interim_Sd_final = keep_only_highest(interim_Sd, 100, Facets.SOCIAL_DISRUPTION)

In [55]:
interim_Sd_final

,created_at,id,source,is_quote_tweet,quoted_tweet_id,in_reply_to_status_id_str,in_reply_to_user_id_str,in_reply_to_screen_name,lang,quote_count,...,QT_place_country_code,QT_coordinates,QT_text,QT_full_text,TEST_FLAG,RT,FINAL_TEXT,final_score,final_keyword_match,FACET
0,Tue Aug 18 20:16:18 +0000 2020,1.2958168150803535e+18,"<a href=""https://mobile.twitter.com"" rel=""nofo...",False,NaN,1.2955387738250732e+18,36130691.0,StriderElass,en,0.0,...,NaN,NaN,NaN,NaN,Y,False,striderelass oh look anoth fuck idiot doesnt r...,100,travel,Sd
1,Tue Aug 18 20:16:21 +0000 2020,1.2958168292445184e+18,"<a href=""https://mobile.twitter.com"" rel=""nofo...",False,NaN,1.2954763582110843e+18,296948244.0,dastrategy,en,0.0,...,NaN,NaN,NaN,NaN,Y,False,cnn cnn regard the interview possibl incent wo...,100,larg,Sd
2,Tue Aug 18 20:16:28 +0000 2020,1.2958168570443612e+18,"<a href=""https://mobile.twitter.com"" rel=""nofo...",False,NaN,NaN,NaN,NaN,en,0.0,...,NaN,NaN,NaN,NaN,Y,False,frustrat lack extens travel credit set to expi...,100,travel,Sd


# 5_Purpose

In [56]:
interim_Purp = data.apply(fuzzy_logic, axis = 1, args = (FINAL_COL_NAME, keywords_Purp, Facets.PURPOSE))
interim_Purp_final = keep_only_highest(interim_Purp, 100, Facets.PURPOSE)

In [57]:
interim_Purp_final

,created_at,id,source,is_quote_tweet,quoted_tweet_id,in_reply_to_status_id_str,in_reply_to_user_id_str,in_reply_to_screen_name,lang,quote_count,...,QT_place_country_code,QT_coordinates,QT_text,QT_full_text,TEST_FLAG,RT,FINAL_TEXT,final_score,final_keyword_match,FACET
0,Tue Aug 18 20:16:18 +0000 2020,1.295816816883835e+18,"<a href=""http://twitter.com/download/iphone"" r...",False,NaN,1.2957868603300618e+18,7.986988832494756e+17,rickgtz22,en,0.0,...,NaN,NaN,NaN,NaN,Y,False,rickgtz22 richard do prevent death do lock for...,100,save,Purp
1,Tue Aug 18 20:16:27 +0000 2020,1.2958168537056256e+18,"<a href=""https://mobile.twitter.com"" rel=""nofo...",False,NaN,NaN,NaN,NaN,en,0.0,...,NaN,NaN,NaN,NaN,Y,False,the nurs work to save coronaviru denier,100,save,Purp


# 6_Positive Emotion

In [58]:
interim_Pe = data.apply(fuzzy_logic, axis = 1, args = (FINAL_COL_NAME, keywords_Pe, Facets.POSITIVE_EMOTION))
interim_Pe_final = keep_only_highest(interim_Pe, 100, Facets.POSITIVE_EMOTION)

In [59]:
interim_Pe_final

,created_at,id,source,is_quote_tweet,quoted_tweet_id,in_reply_to_status_id_str,in_reply_to_user_id_str,in_reply_to_screen_name,lang,quote_count,...,QT_place_country_code,QT_coordinates,QT_text,QT_full_text,TEST_FLAG,RT,FINAL_TEXT,final_score,final_keyword_match,FACET
0,Tue Aug 18 20:16:16 +0000 2020,1.2958168067084124e+18,"<a href=""http://twitter.com/download/android"" ...",False,NaN,1.295814923734397e+18,200661525.0,Georgia_Levan,en,0.0,...,NaN,NaN,NaN,NaN,Y,False,georgialevan sorri correct didnt read wrote ma...,100,safe,Pe
1,Tue Aug 18 20:16:17 +0000 2020,1.2958168129747395e+18,"<a href=""https://mobile.twitter.com"" rel=""nofo...",False,NaN,NaN,NaN,NaN,en,0.0,...,NaN,NaN,NaN,NaN,Y,False,enhanc safeti precaut take to ensur the safeti...,100,safe,Pe
2,Tue Aug 18 20:16:18 +0000 2020,1.2958168154869637e+18,"<a href=""https://mobile.twitter.com"" rel=""nofo...",False,NaN,1.2957398330134897e+18,3231412925.0,liliana_dorina,en,0.0,...,NaN,NaN,NaN,NaN,Y,False,lilianadorina good even dear liliana japan eco...,100,hope,Pe
3,Tue Aug 18 20:16:23 +0000 2020,1.2958168363621827e+18,"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,NaN,en,0.0,...,NaN,NaN,NaN,NaN,Y,False,imagin wed right the govern idea stop the stay...,100,safe,Pe
4,Tue Aug 18 20:16:28 +0000 2020,1.2958168556727624e+18,"<a href=""http://twitter.com/download/iphone"" r...",False,NaN,1.2957891602597806e+18,7.581015198919148e+17,BullPup2A,en,0.0,...,NaN,NaN,NaN,NaN,Y,False,bullpup2a safe space like bunkerboytrump truth...,100,safe,Pe


### All Proportions

In [60]:
print("For IMPLEMENTATION:", proportion(interim_Imp_final, interim_Imp))
print("For ADAPTATION:", proportion(interim_Ada_final, interim_Ada))
print("For NEGATIVE EMOTIONS:", proportion(interim_Ne_final, interim_Ne))
print("For SOCIAL DISRUPTION:", proportion(interim_Sd_final, interim_Sd))
print("For PURPOSE:", proportion(interim_Purp_final, interim_Purp))
print("For POSITIVE EMOTION:", proportion(interim_Pe_final, interim_Pe))

For IMPLEMENTATION: 0.23
For ADAPTATION: 0.05
For NEGATIVE EMOTIONS: 0.07
For SOCIAL DISRUPTION: 0.03
For PURPOSE: 0.02
For POSITIVE EMOTION: 0.05


In [61]:
merged_data = pd.concat([interim_Imp_final, interim_Ada_final, interim_Ne_final, interim_Sd_final, interim_Purp_final, interim_Pe_final])

In [62]:
merged_data

,created_at,id,source,is_quote_tweet,quoted_tweet_id,in_reply_to_status_id_str,in_reply_to_user_id_str,in_reply_to_screen_name,lang,quote_count,...,QT_place_country_code,QT_coordinates,QT_text,QT_full_text,TEST_FLAG,RT,FINAL_TEXT,final_score,final_keyword_match,FACET
0,Tue Aug 18 20:16:16 +0000 2020,1.295816806842749e+18,"<a href=""http://twitter.com/download/iphone"" r...",False,NaN,1.295808857885868e+18,204458518.0,briduimhaoluala,en,0.0,...,NaN,NaN,NaN,NaN,Y,False,briduimhaoluala focu highrisk member mere unio...,100,social distanc,Imp
1,Tue Aug 18 20:16:16 +0000 2020,1.2958168079290737e+18,"<a href=""http://instagram.com"" rel=""nofollow"">...",False,NaN,NaN,NaN,NaN,en,0.0,...,NaN,NaN,NaN,NaN,Y,False,today mission tough quarantinelif keep kill wi...,100,quarantin,Imp
2,Tue Aug 18 20:16:16 +0000 2020,1.2958168082771722e+18,"<a href=""http://twitter.com/download/iphone"" r...",False,NaN,NaN,NaN,NaN,en,0.0,...,NaN,NaN,NaN,NaN,Y,False,sampp 500 close at record high eras the last s...,100,shutdown,Imp
3,Tue Aug 18 20:16:17 +0000 2020,1.2958168101813327e+18,"<a href=""http://twitter.com/download/iphone"" r...",False,NaN,1.2958154284308685e+18,3162911248.0,mcm_ct,en,0.0,...,NaN,NaN,NaN,NaN,Y,False,mcmct anyon spend 2000 ppe shouldnt teach anyon,100,hou,Imp
4,Tue Aug 18 20:16:17 +0000 2020,1.2958168118465413e+18,"<a href=""http://twitter.com/download/iphone"" r...",False,NaN,NaN,NaN,NaN,en,0.0,...,NaN,NaN,NaN,NaN,Y,False,caught glimps someon wear dangl ear mask thoug...,100,hou,Imp
5,Tue Aug 18 20:16:18 +0000 2020,1.2958168135324672e+18,"<a href=""http://twitter.com/download/android"" ...",False,NaN,1.295784486949462e+18,7.810050522070385e+17,MyJoze,en,0.0,...,NaN,NaN,NaN,NaN,Y,False,myjoz trump elimin the entir pandem respons team,100,respon,Imp
6,Tue Aug 18 20:16:18 +0000 2020,1.29581681317607e+18,"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,NaN,en,0.0,...,NaN,NaN,NaN,NaN,Y,False,at home took time out to go to art school part...,100,home,Imp
7,Tue Aug 18 20:16:18 +0000 2020,1.2958168153864028e+18,"<a href=""https://mobile.twitter.com"" rel=""nofo...",False,NaN,1.2958148671448023e+18,1.098415694021124e+18,collegemasc,en,0.0,...,NaN,NaN,NaN,NaN,Y,False,collegemasc 1 start septemb would spread globa...,100,quarantin,Imp
8,Tue Aug 18 20:16:18 +0000 2020,1.295816816883835e+18,"<a href=""http://twitter.com/download/iphone"" r...",False,NaN,1.2957868603300618e+18,7.986988832494756e+17,rickgtz22,en,0.0,...,NaN,NaN,NaN,NaN,Y,False,rickgtz22 richard do prevent death do lock for...,100,respon,Imp
9,Tue Aug 18 20:16:19 +0000 2020,1.295816819010306e+18,"<a href=""http://twitter.com/download/iphone"" r...",False,NaN,1.2958046937430467e+18,1673526829.0,md_mszubairi,en,0.0,...,NaN,NaN,NaN,NaN,Y,False,mdmszubairi share concern run real risk creat ...,100,respon,Imp


In [69]:
merged_data['user_id'].nunique()

37